[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/huggingface/education-toolkit/blob/main/02_ml-demos-with-gradio.ipynb)



💡 **Welcome!**

We’ve assembled a toolkit that university instructors and organizers can use to easily prepare labs, homework, or classes. The content is designed in a self-contained way such that it can easily be incorporated into the existing curriculum. This content is free and uses widely known Open Source technologies (`transformers`, `gradio`, etc).

Alternatively, you can request for someone on the Hugging Face team to run the tutorials for your class via the [ML demo.cratization tour](https://huggingface2.notion.site/ML-Demo-cratization-tour-with-66847a294abd4e9785e85663f5239652) initiative!

You can find all the tutorials and resources we’ve assembled [here](https://huggingface2.notion.site/Education-Toolkit-7b4a9a9d65ee4a6eb16178ec2a4f3599). 

# Tutorial: Build and Host Machine Learning Demos with Gradio ⚡ & Hugging Face 🤗 

**Learning goals:** The goal of this tutorial is to learn How To

1. Build a quick demo for your machine learning model in Python using the `gradio` library
2. Host the demos for free with Hugging Face Spaces
3. Add your demo to the Hugging Face org for your class or conference. This includes:
  *   A setup step for instructors (or conference organizers)
  *   Upload instructions for students (or conference participants)

**Duration**: 20-40
 minutes

**Prerequisites:** Knowledge of Python and basic familiarity with machine learning 


**Author**: [Abubakar Abid](https://twitter.com/abidlabs) (feel free to ping me with any questions about this tutorial) 

All of these steps can be done for free! All you need is an Internet browser and a place where you can write Python 👩‍💻

## Why Demos?

**Demos** of machine learning models are an increasingly important part of machine learning _courses_ and _conferences_. Demos allow:

* model developers to easily **present** their work to a wide audience
* increase **reproducibility** of machine learning research
* diverse users to more easily **identify and debug** failure points of models


As a quick example of what we would like to build, check out the [Keras Org on Hugging Face](https://huggingface.co/keras-io), which includes a description card and a collection of Models and Spaces built by Keras community. Any Space can be opened in your browser and you can use the model immediately, as shown here: 

![](https://i.ibb.co/7y6DGjB/ezgif-5-cc52b7e590.gif)




## 1. Build Quick ML Demos in Python Using the Gradio Library

`gradio` is a handy Python library that lets you build web demos simply by specifying the list of input and output **components** expected by your machine learning model. 

What do I mean by input and output components? Gradio comes with a bunch of predefined components for different kinds of machine learning models. Here are some examples:

* For an **image classifier**, the expected input type is an `Image` and the output type is a `Label`. 
* For a **speech recognition model**, the expected input component is an `Microphone` (which lets users record from the browser) or `Audio` (which lets users drag-and-drop audio files), while the output type is `Text`. 
* For a **question answering model**, we expect **2 inputs**: [`Text`, `Text`], one textbox for the paragraph and one for the question, and the output type is a single `Text` corresponding to the answer. 

You get the idea... (for all of the supported components, [see the docs](https://gradio.app/docs/))

In addition to the input and output types, Gradio expects a third parameter, which is the prediction function itself. This parameter can be ***any* regular Python function** that takes in parameter(s) corresponding to the input component(s) and returns value(s) corresponding to the output component(s)

Enough words. Let's see some code!

In [ ]:
# First, install Gradio
!pip install --quiet gradio

     |████████████████████████████████| 871 kB 5.1 MB/s 
     |████████████████████████████████| 2.0 MB 41.5 MB/s 
     |████████████████████████████████| 52 kB 787 kB/s 
     |████████████████████████████████| 1.1 MB 25.8 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 210 kB 56.5 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 271 kB 58.7 MB/s 
     |████████████████████████████████| 144 kB 58.8 MB/s 
     |████████████████████████████████| 10.9 MB 44.8 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
     |████████████████████████████████| 79 kB 6.6 MB/s 
     |████████████████████████████████| 856 kB 60.6 MB/s 
     |████████████████████████████████| 61 kB 374 kB/s 
     |████████████████████████████████| 3.6 MB 50.0 MB/s 
     |████████████████████████████████| 58 kB 4.5 MB/s 


In [1]:
import numpy as np

def sepia(image):
    sepia_filter = np.array(
        [[0.393, 0.769, 0.189], 
         [0.349, 0.686, 0.168], 
         [0.272, 0.534, 0.131]]
    )
    sepia_img = image.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

In [ ]:
import gradio as gr

# Write 1 line of Python to create a simple GUI
gr.Interface(fn=sepia, inputs="image", outputs="image").launch(share=True);

Running the code above should produce a simple GUI inside this notebook allowing you to type example inputs and see the output returned by your function. 

Notice that we define an `Interface` using the 3 ingredients mentioned earlier:
* A function
* Input component(s)
* Output component(s)

This is a simple example for images, but the same principle holds true for any other kind of data type. For example, here is an interface that generates a musical tone when provided a few different parameters (the specific code inside `generate_tone()` is not important for the purpose of this tutorial):

In [ ]:
import numpy as np
import gradio as gr

def generate_tone(note, octave, duration):
    sampling_rate = 48000
    a4_freq, tones_from_a4 = 440, 12 * (octave - 4) + (note - 9)
    frequency = a4_freq * 2 ** (tones_from_a4 / 12)
    audio = np.linspace(0, int(duration), int(duration) * sampling_rate)
    audio = (20000 * np.sin(audio * (2 * np.pi * frequency))).astype(np.int16)
    return sampling_rate, audio

gr.Interface(
    generate_tone,
    [
        gr.inputs.Dropdown(["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"], type="index"),
        gr.inputs.Slider(4, 6, step=1),
        gr.inputs.Textbox(type="number", default=1, label="Duration in seconds"),
    ],
    "audio",
    title="Generate a Musical Tone!"
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://20619.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f84a8c6f850>,
 'http://127.0.0.1:7860/',
 'https://20619.gradio.app')

**Challenge #1**: build a Gradio demo that takes in an image and returns the same image *flipped upside down* in less than 10 lines of Python code.

There are a lot more examples you can try in Gradio's [getting started page](https://gradio.app/getting_started/), which cover additional features such as:
* Adding example inputs
* Adding _state_ (e.g. for chatbots)
* Sharing demos easily using one parameter called `share` (<-- this is pretty cool 😎)

It is especially easy to demo a `transformers` model from Hugging Face's Model Hub, using the special `gr.Interface.load` method. 

Let's try a text-to-speech model built by Facebook:

In [ ]:
import gradio as gr

gr.Interface.load("huggingface/facebook/fastspeech2-en-ljspeech").launch(share=True);

Here is the code to build a demo for [GPT-J](https://huggingface.co/EleutherAI/gpt-j-6B), a large language model & add a couple of examples inputs:

In [ ]:
import gradio as gr

examples = [["The Moon's orbit around Earth has"], ["There once was a pineapple"]]

gr.Interface.load("huggingface/EleutherAI/gpt-j-6B", examples=examples).launch(share=True);

Fetching model from: https://huggingface.co/EleutherAI/gpt-j-6B
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://30262.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


**Challenge #2**: Go to the [Hugging Face Model Hub](https://huggingface.co/models), and pick a model that performs one of the other tasks supported in the `transformers` library (other than the two you just saw: text generation or text-to-speech). Create a Gradio demo for that model using `gr.Interface.load`.

## 2. Host the Demo (for free) on Hugging Face Spaces

Once you made a Gradio demo, you can host it permanently on Hugging Spaces very easily:

Here are the steps to that (shown in the GIF below):

A. First, create a Hugging Face account if you do not already have one, by visiting https://huggingface.co/ and clicking "Sign Up"

B. Once you are logged in, click on your profile picture and then click on "New Space" underneath it to get to this page: https://huggingface.co/new-space

C. Give your Space a name and a license. Select "Gradio" as the Space SDK, and then choose "Public" if you are fine with everyone accessing your Space and the underlying code

D. Then you will find a page that provides you instructions on how to upload your files into the Git repository for that Space. You may also need to add a `requirements.txt` file to specify any Python package dependencies.

E. Once you have pushed your files, that's it! Spaces will automatically build your Gradio demo allowing you to share it with anyone, anywhere!

![GIF](https://huggingface.co/blog/assets/28_gradio-spaces/spaces-demo-finalized.gif)





You can even embed your Gradio demo on any website -- in a blog, a portfolio page, or even in a colab notebook, like I've done with a Pictionary sketch recognition model below:

In [ ]:
from IPython.display import IFrame
IFrame(src='https://hf.space/gradioiframe/abidlabs/Draw/+', width=1000, height=800)

**Challenge #3**: Upload your Gradio demo to Hugging Face Spaces and get a permanent URL for it. Share the permanent URL with someone (a colleague, a collaborator, a friend, a user, etc.) -- what kind of feedback do you get on your machine learning model?

## 3. Add your demo to the Hugging Face org for your class or conference

#### **Setup** (for instructors or conference organizers)

A. First, create a Hugging Face account if you do not already have one, by visiting https://huggingface.co/ and clicking "Sign Up"

B. Once you are logged in, click on your profile picture and then click on "New Organization" underneath it to get to this page: https://huggingface.co/organizations/new

C. Fill out the information for your class or conference. We recommend creating a separate organization each time that a class is taught (for example, "Stanford-CS236g-20222") and for each year of the conference.

D. Your organization will be created and now now users will be able request adding themselves to your organizations by visiting the organization page.

E. Optionally, you can change the settings by clicking on the "Organization settings" button. Typically, for classes and conferences, you will want to navigate to `Settings > Members` and set the "Default role for new members" to be "write", which allows them to submit Spaces but not change the settings. 

#### For students or conference participants

A. Ask your instructor / coneference organizer for the link to the Organization page if you do not already have it

B. Visit the Organization page and click "Request to join this org" button, if you are not yet part of the org.

C. Then, once you have been approved to join the organization (and built your Gradio Demo and uploaded it to Spaces -- see Sections 1 and 2), then simply go to your Space and go to `Settings > Rename or transfer this space` and then select the organization name under `New owner`. Click the button and the Space will now be added to your class or conference Space!   